In [1]:
import sys
sys.path.append("/Users/anuram/Documents/hydrogenase-ftir/src")
sys.path.append("/Users/anuram/Documents/hydrogenase-ftir/notebooks/anu_winter_2024")

In [3]:
%load_ext autoreload
%autoreload 2

#Setting Up and Importing the Necessary Packages/Libraries
##Package for reading in Bruker OPUS type files
from brukeropusreader import read_file
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import pathlib
import numpy as np
import pandas as pd
#Local Functions
from hydrogenase_processing.cut_range import cut_range_subtraction
from hydrogenase_processing.second_deriv import second_deriv, first_deriv
#testing
#find peaks
from scipy.signal import find_peaks, peak_widths
from hydrogenase_processing.anchor_points import get_peaks, get_start_end_anchorpoints, get_all_anchor_points, baseline_spline, get_peaks_absorbance, baseline_correction, get_peak_baseline_absorbance, plot_baseline_data
from hydrogenase_processing.peak_fit import gaussian, peak_fit, lorentzian

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#Paths to Local Data
path_to_water_vapor_data = pathlib.Path("../../data/opus_files/water_vapor")
path_to_output_plots_= pathlib.Path("../../data/output_plots")
path_to_all_test_data = pathlib.Path("../../data/opus_files/subtraction_coefficient_testing") 

## Importing the Data form the local paths specified above ##

In [17]:
#Importing Water Vapor
wv_data = read_file(f'{path_to_water_vapor_data}''/water vapor 4cm-1.1')

In [6]:
#### cant do 176 cos i cant see it 
#Pulling in all test data
second_derivative_test_raw_data = list(path_to_all_test_data.iterdir())
second_derivative_test_raw_data.sort()

#Initializing dict of raw spectra files from the file system
raw_data = dict()

#Populating the raw_test_data dict with all the read in raw opus files
for i in (second_derivative_test_raw_data):
    if not i.name.startswith('.DS_Store'):
        raw_data[i.name[0:4]] = read_file(i)

print(raw_data.keys())

dict_keys(['011a', '011b', '011c', '011d', '011e', '011f', '011g', '011h', '011i', '011k', '011l', '011m', '011n', '011o', '011p', '011q', '011r', '011s', '011t', '160 ', '162 ', '164 ', '165 ', '166 ', '167 ', '168 ', '169 ', '170 ', '171 ', '172 '])


In [11]:
test_comparisons_parameters_df = pd.read_excel("../../data/test_subtraction_coefficients.xlsx", sheet_name="uncut_samples")

#full_file_names = test_comparisons_parameters_df["file_name"]

test_comparisons_parameters_df["file_name"] = test_comparisons_parameters_df["file_name"].apply(lambda file_name: file_name[0:4]) 

#test_comparisons_parameters_df["file_name"] = exp_num_file_name


indexed_test_comparison_parameters = test_comparisons_parameters_df.set_index('file_name')


## Subtracting Water Vapor


In [12]:
#Initializing dict of post water vapor subtraction spectra
cut_range_sub_wv_data = dict()

results = list()

for idx, row in indexed_test_comparison_parameters.iterrows():  
    if idx in raw_data:
        raw_data_i = raw_data[idx]
        cut_range_sub_wv_data[f'{idx}_cut_range_wv_sub'] = cut_range_subtraction(raw_data_i, wv_data, row["range_start"], row["range_end"], SG_poly = 3, SG_points = 21)
        subtraction_parameters = cut_range_sub_wv_data[f'{idx}_cut_range_wv_sub'][0][0].fit_atm_params
        results.append(subtraction_parameters[0])
    else:
        results.append(None)

indexed_test_comparison_parameters["pb_subtraction_coefficient"] = results

2150.0 1800.0
959 1141
2150.0 1800.0
959 1141
2150.0 1800.0
959 1141
2150.0 1800.0
959 1141
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115
2150.0 1850.0
959 1115


# Get Second derivative


In [14]:
print(list(cut_range_sub_wv_data.keys()))


['160 _cut_range_wv_sub', '167 _cut_range_wv_sub', '162 _cut_range_wv_sub', '011a_cut_range_wv_sub', '011b_cut_range_wv_sub', '011c_cut_range_wv_sub', '011d_cut_range_wv_sub', '011e_cut_range_wv_sub', '011f_cut_range_wv_sub', '011g_cut_range_wv_sub', '011h_cut_range_wv_sub', '011i_cut_range_wv_sub', '011k_cut_range_wv_sub', '011l_cut_range_wv_sub', '011m_cut_range_wv_sub', '011n_cut_range_wv_sub', '011o_cut_range_wv_sub', '011p_cut_range_wv_sub', '011q_cut_range_wv_sub', '011r_cut_range_wv_sub', '011s_cut_range_wv_sub', '011t_cut_range_wv_sub', '164 _cut_range_wv_sub', '165 _cut_range_wv_sub', '166 _cut_range_wv_sub', '168 _cut_range_wv_sub', '169 _cut_range_wv_sub', '170 _cut_range_wv_sub', '171 _cut_range_wv_sub', '172 _cut_range_wv_sub']


In [13]:
#Extracting wavenb and abs for one of the corrected spectra
example_cut_sub = cut_range_sub_wv_data["176 _cut_range_wv_sub"]

x = second_deriv(example_cut_sub, show_plots=True)


KeyError: '176 _cut_range_wv_sub'

## Testing with the smaller peak width algorithm and tuning the ajustment factor

In [ ]:
start = 1
end = 5
step = 0.25
anchor_point_dict = {}
adjustment_factor = np.arange(start, end + step, step)
#adjustment_factor = [1]
#using default prominence threshold of 15%
for adj_factor in adjustment_factor:
    plt.figure()
    peaks_index, deriv_x_peak_val = get_peaks(x)

    wv_startIdx, wv_endIdx = get_start_end_anchorpoints(peaks_index[0], x)
    y_corr_abs = example_cut_sub[0][0].sub_spectrum
    anchor_points_raw_data = example_cut_sub[0][0].wavenb
    anchor_point_dict[round(adj_factor, 2)] =get_all_anchor_points(wv_startIdx, wv_endIdx, deriv_x_peak_val, anchor_points_raw_data, y_corr_abs, f'Anchor points using {round(adj_factor, 2)} as adj_factor', round(adj_factor, 2))

In [ ]:
anchor_points = anchor_point_dict.get(2.25)

baseline_curve = baseline_spline(anchor_points)
peak_wv, peak_abs = get_peaks_absorbance(deriv_x_peak_val, anchor_points_raw_data, y_corr_abs)
plt.plot(anchor_points_raw_data, y_corr_abs, label = 'raw data')
plt.plot(baseline_curve['wavenumber'], baseline_curve['absorbance'], 'r--', label = 'baseline spline')
plt.plot(peak_wv, peak_abs, 'go', label = 'peaks')
plt.legend()